In [3]:
import sys
sys.path.append('../scripts/')
from utility import read_file, create_folder, temp_record_query, temp_record_sdf

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [5]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/10/01 23:11:19 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.12 instead (on interface en0)
22/10/01 23:11:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/01 23:11:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1. Connect Transactions and Merchants AS `sdf`

In [6]:
print("================ Transactions ================")
# transactions_sdf = read_file(spark, 'transactions_20210228_20210827_snapshot')
transactions_sdf = read_file(spark, 'transactions_20210828_20220227_snapshot')
# transactions_sdf = read_file(spark, 'transactions_20220228_20220828_snapshot')
temp = transactions_sdf.dropna(how='any').count()
print( f'There are {transactions_sdf.count()} data, and {temp} of them have no null value.')

================ Transactions ================
|> Loading File...


|> Loading Finished!


-RECORD 0----------------------------------------------
 user_id        | 14935                                
 merchant_abn   | 79417999332                          
 dollar_value   | 136.06570809815838                   
 order_id       | 23acbb7b-cf98-4580-9775-86b8e0a2bd88 
 order_datetime | 2021-11-26                           
only showing top 1 row



There are 4508106 data, and 4508106 of them have no null value.


In [7]:
print("================ Merchant ================")
merchants_sdf = read_file(spark, 'merchants_data.parquet', '../data/curated/', truncate=20)
temp = merchants_sdf.dropna(how='any').count()
print( f'There are {merchants_sdf.count()} data, and {temp} of them have no null value.')

================ Merchant ================
|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 merchant_name | Felis Limited        
 tags          | furniture, home f... 
 merchant_abn  | 10023283211          
 take_rate     | 0.18                 
 type          | e                    
only showing top 1 row

There are 4026 data, and 4026 of them have no null value.


- Known that Transactions and Merchants table have no null value

In [8]:
sdf = transactions_sdf.join(merchants_sdf, how='left', on='merchant_abn')
temp = sdf.dropna(how='any', subset=['merchant_name']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 4508106 data, and 4323692 of them have no null value.


- After connect the Transactions and Merchants Table by `merchant_abn`, we find that some Transactions not from the merchants in the table. Because they are not in our list of rank therefore, delete those null value.

In [7]:
# sdf = sdf.dropna(how='any', subset=['merchant_name'])

- There are 4026 merchants, and for `transactions_20210228_20210827_snapshot` have 3643266 records. 

    After combination, there are **3494038** rest. **3643266 - 3494038 = 149228** records has been deleted. 
    
    `149228 / 3643266 = 0.04095995186736297`

In [9]:
sdf.show(5)

+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
|merchant_abn|user_id|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|
+------------+-------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+
| 79417999332|  14935|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|
| 46451548968|      1| 72.61581642788431|76bab304-fa2d-400...|    2021-11-26|Tempus Eu Ligula ...|health and beauty...|     6.04|   a|
| 89518629617|  14936|3.0783487174439297|a2ae446a-2959-41c...|    2021-11-26|Vulputate Velit E...|tent and awning s...|     3.09|   c|
| 49167531725|      1| 51.58228625503599|7080c274-17f7-4cc...|    2021-11-26|     Felis Institute|digital goods: bo...|     6.42|   a|
| 31101120643|  14936|25.228114942417797|8e301c0f-06ab-

In [10]:
null_df = sdf.filter(sdf.merchant_name.isNull())
# 184414
null_df.count()
# 381
new_merchant = null_df.drop_duplicates(['merchant_abn'])

### 2. `sdf` join with consumer

In [11]:
print("================ Customer ================")
consumer = read_file(spark, 'new_consumer_data.parquet', '../data/curated/')
temp = consumer.dropna(how='any').count()
print( f'There are {consumer.count()} data, and {temp} of them have no null value.')

================ Customer ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------------------
 consumer_id | 680810                     
 name        | Courtney Mendez            
 address     | 802 Knapp Harbors Apt. 769 
 state       | QLD                        
 postcode    | 9013                       
 gender      | Female                     
 user_id     | 71144                      
only showing top 1 row

There are 499999 data, and 499999 of them have no null value.


- consumer join with poa population data

In [12]:
print("================  ================")
poa_sdf = read_file(spark, 'external_pos_population.parquet', '../data/curated/')

================  ================
|> Loading File...
|> Loading Finished!


-RECORD 0--------------------------------------------------------------------------------------
 postcode   | 6935                                                                             
 POA_CODE21 | 6055                                                                             
 geometry   | [01 03 00 00 00 01 00 00 00 F2 05 00 00 68 A7 F7 0B 98 FD 5C 40 C0 AB 6F 9D 5... 
 2020(no.)  | 33667.56962451867                                                                
 2021(no.)  | 35471.46982550462                                                                
only showing top 1 row



In [13]:
poa_sdf = poa_sdf.drop('geometry')
poa_sdf = poa_sdf.withColumnRenamed('2020(no.)', '2020_popu').withColumnRenamed('2021(no.)', '2021_popu')

In [14]:
poa_sdf.show(5)

+--------+----------+-----------------+-----------------+
|postcode|POA_CODE21|        2020_popu|        2021_popu|
+--------+----------+-----------------+-----------------+
|    6935|      6055|33667.56962451867|35471.46982550462|
|    1109|      2000|31245.01595130939|29155.59319547952|
|    6849|      6014|23594.15360007898|23626.47316732434|
|    1114|      2000|31245.01595130939|29155.59319547952|
|    1825|      2144|42033.87774893655|40303.38192096321|
+--------+----------+-----------------+-----------------+
only showing top 5 rows



In [15]:
temp = poa_sdf.dropna(how='any').count()
print( f'There are {poa_sdf.count()} data, and {temp} have no null value.')

There are 3158 data, and 3158 have no null value.


In [16]:
consumer = consumer.join(poa_sdf, on='postcode', how='left')

In [17]:
consumer.show(20)

+--------+-----------+--------------------+--------------------+-----+-----------+-------+----------+-----------------+-----------------+
|postcode|consumer_id|                name|             address|state|     gender|user_id|POA_CODE21|        2020_popu|        2021_popu|
+--------+-----------+--------------------+--------------------+-----+-----------+-------+----------+-----------------+-----------------+
|    3341|     824277|     Jessica Perkins|   004 Joseph Stream|  VIC|     Female|  71164|      3341|3837.071226952807|3949.498607935793|
|    3612|     240015|           Paul Bond|0126 Copeland Hil...|  VIC|Undisclosed|  71150|      3612|1261.665090031683|1280.963899794179|
|    3919|    1089439|       Michael Frank|515 Martinez Harb...|  VIC|       Male|  71157|      3919|1456.085532548059|1461.088434365399|
|    4006|     360950|          Scott Diaz|10879 Warren Trai...|  QLD|       Male|  71160|      4006|29285.45099563307|29731.72434655262|
|    1108|     257426|          Ev

In [18]:
consumer.dropna(how='any').count()

498587

In [19]:
# 有9个postcode没有找到对应的数据
# 就是做预处理找过的 --- 保留到fraud做处理
consumer[consumer.POA_CODE21.isNull()].drop_duplicates(['postcode'])

postcode,consumer_id,name,address,state,gender,user_id,POA_CODE21,2020_popu,2021_popu
1441,341498,Courtney Alexander,82265 Hardy Hill,NSW,Female,77783,null,null,null
1476,588776,Elizabeth Sanchez,343 Middleton Shoal,NSW,Female,73404,null,null,null
1819,1131147,Marc Wilson,162 Shields Inlet...,NSW,Male,72313,null,null,null
2013,861495,Erin Rodriguez,707 Xavier Brook,NSW,Female,75224,null,null,null
3399,724573,Andrea Stone,627 Christine Cor...,VIC,Female,72496,null,null,null
3989,1093754,Tracey Williams,49197 Nunez Burg,VIC,Female,71746,null,null,null
6435,1217055,Douglas Jones,71897 Kristina Hi...,WA,Undisclosed,71334,null,null,null
8785,806615,Destiny Morgan,68138 Welch Divid...,VIC,Female,72252,null,null,null
9999,545212,Daniel Walker,2217 Fleming Rest...,VIC,Male,72597,null,null,null


In [20]:
consumer = consumer.na.fill(subset='2021_popu', value = 0)

In [22]:
consumer = consumer.na.fill(subset='POA_CODE21', value = 0)

In [24]:
consumer[consumer.POA_CODE21.isNull()].drop_duplicates(['postcode'])

postcode,consumer_id,name,address,state,gender,user_id,POA_CODE21,2020_popu,2021_popu


- join


In [25]:
sdf

merchant_abn,user_id,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type
79417999332,14935,136.06570809815838,23acbb7b-cf98-458...,2021-11-26,Phasellus At Company,"gift, card, novel...",4.95,b
46451548968,1,72.61581642788431,76bab304-fa2d-400...,2021-11-26,Tempus Eu Ligula ...,health and beauty...,6.04,a
89518629617,14936,3.0783487174439297,a2ae446a-2959-41c...,2021-11-26,Vulputate Velit E...,tent and awning s...,3.09,c
49167531725,1,51.58228625503599,7080c274-17f7-4cc...,2021-11-26,Felis Institute,digital goods: bo...,6.42,a
31101120643,14936,25.228114942417797,8e301c0f-06ab-45c...,2021-11-26,Commodo Hendrerit...,"cable, satellite,...",6.37,a
67978471888,2,691.5028234458998,0380e9ad-b0e8-420...,2021-11-26,Magna Malesuada C...,artist supply and...,5.56,a
60956456424,14936,102.13952056640888,5ac3da9c-5147-452...,2021-11-26,Ultricies Digniss...,"gift, card, novel...",4.69,b
47644196714,2,644.5220654863093,4e368e44-86f8-4de...,2021-11-26,Auctor Non Feugia...,motor vehicle sup...,1.89,c
39649557865,14938,209.12780951421405,4d78cd01-4bab-494...,2021-11-26,Arcu Morbi Institute,artist supply and...,1.47,c
88402174457,3,141.0387993699113,c50c957d-ecfc-430...,2021-11-26,Duis Dignissim LLC,"furniture, home f...",6.06,a


In [26]:
sdf = sdf.join(consumer, how='left', on='user_id')
temp = sdf.dropna(how='any', subset=['consumer_id']).count()
print( f'There are {sdf.count()} data, and {temp} of them have no null value.')

There are 4508106 data, and 4508106 of them have no null value.


In [27]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|           name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|    4563|    1059280|  Cameron Adams|0280 Carpenter Lodge|  QLD|  Male|      4563|4880.286405609036|4951.48

### 3. `sdf` join with the external dataset  --- ncd

In [28]:
print("================  ================")
ncd_sdf = read_file(spark, 'external_ncd.parquet', '../data/curated/')
temp = ncd_sdf.dropna(how='any').count()
print( f'There are {ncd_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0---------------------
 Date            | 2020-01-25 
 New cases / day | 4          
only showing top 1 row

There are 959 data, and 959 have no null value.


In [29]:
ncd_sdf.show(5)

+----------+---------------+
|      Date|New cases / day|
+----------+---------------+
|2020-01-25|              4|
|2020-01-26|              0|
|2020-01-27|              1|
|2020-01-28|              0|
|2020-01-29|              4|
+----------+---------------+
only showing top 5 rows



In [30]:
sdf = sdf.join(ncd_sdf, sdf.order_datetime == ncd_sdf.Date, 'left')
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+----------+---------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|           name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|      Date|New cases / day|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+----------+---------------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|    4563|    1059280|  Cam

In [31]:
temp = sdf.dropna(how='any', subset=['Date']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

There are 4508106 data, and 4508106 have no null value.


In [32]:
sdf = sdf.drop('Date')

In [29]:
# temp = sdf.to_pandas_on_spark()
# temp[temp.Date.isna()]

### 4. join with external retail trade seasonal

In [33]:
print("================  ================")
rts_sdf = read_file(spark, 'external_retail_trade_seasonal.parquet', '../data/curated/')
temp = rts_sdf.dropna(how='any').count()
print( f'There are {rts_sdf.count()} data, and {temp} have no null value.')

================  ================
|> Loading File...
|> Loading Finished!
-RECORD 0----------------------------------------------
 month                                    | 2022-04-01 
 food                                     | 13182.7    
 household goods                          | 6023.3     
 clothing footwear and personal accessory | 2912.8     
 department store                         | 1782.5     
 other                                    | 5317.8     
 cafes restaurants and takeway foods      | 4704       
 total                                    | 33923.1    
only showing top 1 row

There are 18 data, and 18 have no null value.


In [34]:
rts_sdf.printSchema()

root
 |-- month: date (nullable = true)
 |-- food: string (nullable = true)
 |-- household goods: string (nullable = true)
 |-- clothing footwear and personal accessory: string (nullable = true)
 |-- department store: string (nullable = true)
 |-- other: string (nullable = true)
 |-- cafes restaurants and takeway foods: string (nullable = true)
 |-- total: string (nullable = true)



In [35]:
rts_sdf = rts_sdf.select(['month', 'total'])

In [36]:
sdf = sdf.join(rts_sdf, (F.months_between(sdf.order_datetime, rts_sdf.month) < 1) & (F.months_between(sdf.order_datetime, rts_sdf.month) >= 0), 'left')

In [37]:
temp = sdf.dropna(how='any', subset=['month']).count()
print( f'There are {sdf.count()} data, and {temp} have no null value.')

There are 4508106 data, and 4508106 have no null value.


In [38]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|           name|             address|state|gender|POA_CODE21|        2020_popu|        2021_popu|New cases / day|     month|  total|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+-----------------+---------------+----------+-------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|  

In [39]:
sdf = sdf.drop('2020_popu')

In [40]:
sdf[sdf.POA_CODE21.isNull()].drop_duplicates(['postcode'])

22/10/01 23:17:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,New cases / day,month,total


In [41]:
sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+---------------+----------+-------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|take_rate|type|postcode|consumer_id|           name|             address|state|gender|POA_CODE21|        2021_popu|New cases / day|     month|  total|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+---------+----+--------+-----------+---------------+--------------------+-----+------+----------+-----------------+---------------+----------+-------+
|  14935| 79417999332|136.06570809815838|23acbb7b-cf98-458...|    2021-11-26|Phasellus At Company|gift, card, novel...|     4.95|   b|    4563|    1059280|  Cameron Adams|0280 Carpenter Lodg

In [ ]:
# from pyspark.sql.functions import datediff, months_between

# # 1.日期差
# df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
# df.select(datediff(df.d1, df.d2).alias('diff')).show()

# # 2. 月份差
# df = spark.createDataFrame([('1999-02-28 10:30:00', '1996-10-30')], ['t', 'd'])
# df.select(months_between(df.t, df.d).alias('months')).show()

In [42]:
path = '../data/curated/data'
if (create_folder(path)):
    sdf.write.partitionBy('order_datetime').parquet(path, mode='append')
else:
    sdf.write.partitionBy('order_datetime').parquet(path, mode='overwrite')


|> The folder already exist!
|> Files already exist under this folder:
   ['order_datetime=2021-10-19', 'order_datetime=2021-10-26', 'order_datetime=2021-10-21', 'order_datetime=2021-10-28', 'order_datetime=2021-10-17', 'order_datetime=2021-10-10', 'order_datetime=2022-02-24', 'order_datetime=2022-01-03', 'order_datetime=2022-01-04', 'order_datetime=2022-02-23', 'order_datetime=2022-02-15', 'order_datetime=2022-02-12', 'order_datetime=2021-10-11', 'order_datetime=2021-10-29', 'order_datetime=2021-10-16', 'order_datetime=2021-10-20', 'order_datetime=2021-10-18', 'order_datetime=2021-10-27', 'order_datetime=2022-02-13', 'order_datetime=2022-02-14', 'order_datetime=2022-02-22', 'order_datetime=2022-01-05', 'order_datetime=2022-01-02', 'order_datetime=2022-02-25', 'order_datetime=2021-09-25', 'order_datetime=2021-12-09', 'order_datetime=2021-11-11', 'order_datetime=2021-11-29', 'order_datetime=2021-09-22', 'order_datetime=2021-11-16', 'order_datetime=2021-12-31', 'order_datetime=2021-09-14

In [43]:
sdf

user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,take_rate,type,postcode,consumer_id,name,address,state,gender,POA_CODE21,2021_popu,New cases / day,month,total
7,62224020443,212.60670786220882,3dfa8cb9-b785-40d...,2021-12-21,Hendrerit A Corpo...,"watch, clock, and...",6.64,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"4,522",2021-12-01,31967.9
7,99275601275,111.48028697136225,58af5b18-f28b-464...,2021-10-14,Proin Velit Sed F...,"furniture, home f...",1.87,c,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"2,683",2021-10-01,31132.7
7,26749181046,154.59228188209104,8673c0d9-1515-487...,2021-10-14,Neque Tellus Inco...,"watch, clock, and...",3.34,b,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"2,683",2021-10-01,31132.7
7,63685007785,63.33801058439837,a5962e5d-ab47-431...,2021-10-14,Augue Eu Tempor A...,"cable, satellite,...",0.38,e,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"2,683",2021-10-01,31132.7
7,88435572170,43.22350867054218,abaae4de-a0ff-4b1...,2021-09-10,Et LLP,"gift, card, novel...",2.84,c,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"1,866",2021-09-01,29756.7
7,52065114842,96.79882878080512,04a5a999-4756-4b3...,2021-09-09,Gravida Non PC,digital goods: bo...,3.98,b,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"1,727",2021-09-01,29756.7
7,82065156333,27.987127270166056,f3149d19-98df-4fc...,2021-09-09,Nascetur Ridiculu...,tent and awning s...,5.88,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"1,727",2021-09-01,29756.7
7,43698901053,504.8594541188235,f58962c7-4d0d-4bd...,2022-02-07,Pede Praesent Ltd,tent and awning s...,5.98,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"20,276",2022-02-01,33085.4
7,76314317957,11.774892752790667,0b6a695c-503d-432...,2022-01-14,Semper Corp.,florists supplies...,5.93,a,4606,511685,Andrea Jones,122 Brandon Cliff,QLD,Female,4606,557.0307901493427,"66,635",2022-01-01,32492
14935,79417999332,136.06570809815838,23acbb7b-cf98-458...,2021-11-26,Phasellus At Company,"gift, card, novel...",4.95,b,4563,1059280,Cameron Adams,0280 Carpenter Lodge,QLD,Male,4563,4951.486136674352,"1,621",2021-11-01,33345


In [44]:
sdf.count()

4508106